In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from openpyxl import load_workbook

In [2]:
def append_to_file(new_df):
    file_path = "../data/AirData.xlsx"
    sheet_name = 'AirNow'

    book = load_workbook(file_path)
    sheet = book[sheet_name]
    startrow = sheet.max_row
    
    # Append without setting .book manually
    with pd.ExcelWriter(file_path, engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:
        new_df.to_excel(writer, sheet_name=sheet_name, startrow=startrow, index=False, header=False)

In [3]:
def process_data(url, date):
    print("Processing : " + url)
    air_df = pd.read_csv(url,sep = '|',header = None,
            names =['issuedate','validdate','time','timezone','record_sequence','datatype','primary','reporting_area','state_code','lat','lang','parameter_name','aqi_value','aqi_category','action_day','discussion','forecast_source'])
    airnow_data = air_df.drop(columns= ['issuedate','validdate', 'time','record_sequence', 'datatype','primary',
                                    'action_day','discussion', 'forecast_source'])
    airnow_data.dropna(inplace= True)
    airnow_data = airnow_data.drop(columns=['timezone','aqi_category'])
    airnow_data = airnow_data.loc[airnow_data.groupby(['reporting_area','parameter_name'])['aqi_value'].idxmax().values]
    air = airnow_data.pivot(index= ['reporting_area','state_code','lat','lang'],columns ='parameter_name',values= 'aqi_value').reset_index()
    air['CO'] = air['CO'].round().astype('Int64')
    air['NO2'] = air['NO2'].round().astype('Int64')
    air['OZONE'] = air['OZONE'].round().astype('Int64')
    air['PM10'] = air['PM10'].round().astype('Int64')
    air['PM2.5'] = air['PM2.5'].round().astype('Int64')
    air = air.drop(air.index[1:2])
    air = air[air.state_code.str.strip()!='']
    air = air.reset_index(drop = True)
    air['date'] = pd.to_datetime(date)
    air['date'] = pd.to_datetime(air['date'], format='%Y-%m-%d', errors='coerce')
    air['year'] = air['date'].dt.year
    air['month'] = air['date'].dt.month
    air['weekday'] = air['date'].dt.weekday
    air['month_name'] = air['date'].dt.strftime('%B')
    append_to_file(air)
    return air

In [4]:
for year in range(2019, 2025):
    for month in range(1, 13):
        if month < 10 :
            sMonth = "0"+str(month)
        else:
            sMonth = str(month)
        process_data("https://files.airnowtech.org/airnow/"+str(year)+"/"+str(year)+sMonth+"01/reportingarea.dat", str(year)+"-"+sMonth+"-01")

Processing : https://files.airnowtech.org/airnow/2019/20190101/reportingarea.dat
Processing : https://files.airnowtech.org/airnow/2019/20190201/reportingarea.dat
Processing : https://files.airnowtech.org/airnow/2019/20190301/reportingarea.dat
Processing : https://files.airnowtech.org/airnow/2019/20190401/reportingarea.dat
Processing : https://files.airnowtech.org/airnow/2019/20190501/reportingarea.dat
Processing : https://files.airnowtech.org/airnow/2019/20190601/reportingarea.dat
Processing : https://files.airnowtech.org/airnow/2019/20190701/reportingarea.dat
Processing : https://files.airnowtech.org/airnow/2019/20190801/reportingarea.dat
Processing : https://files.airnowtech.org/airnow/2019/20190901/reportingarea.dat
Processing : https://files.airnowtech.org/airnow/2019/20191001/reportingarea.dat
Processing : https://files.airnowtech.org/airnow/2019/20191101/reportingarea.dat
Processing : https://files.airnowtech.org/airnow/2019/20191201/reportingarea.dat
Processing : https://files.a